In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time 
%matplotlib inline

from sklearn.feature_selection import SelectFromModel

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [49]:
DATA_PATH = "../Data/"

In [50]:
"""
    Step 0 : Get Data
    
"""

def GetData(data_path):
    operation_train = pd.read_csv(data_path + "operation_train_new.csv")
    transaction_train = pd.read_csv(data_path + "transaction_train_new.csv")
    operation_test = pd.read_csv(data_path + "operation_round1_new.csv")
    transaction_test = pd.read_csv(data_path + "transaction_round1_new.csv")
    tag = pd.read_csv(data_path + "tag_train_new.csv")
    return operation_train, transaction_train, tag, operation_test, transaction_test

In [4]:
""" 
    Step 1 : Data Preprocessing 

    Drop Duplicates
    Fill Na
    Drop Outliers: After Data Exploration 
    
"""

# Create tag_test
def CreateTagTest(operation_test, transaction_test):
#     uid_test = pd.concat([operation_test["UID"], transaction_test["UID"]])
    uid_test = pd.concat([operation_test["UID"], transaction_test["UID"]]).unique()
    uid_test = np.sort(uid_test.values)
    tag_test = pd.DataFrame()
    tag_test["UID"] = uid_test
    tag_test["Tag"] = -1
    return tag_test

# Merge Data
def MergeData(operation_train, transaction_train, tag):
    uid_tag_dict = dict(zip(tag["UID"], tag["Tag"]))
    operation_train["tag"] = operation_train["UID"].map(uid_tag_dict)
    transaction_train["tag"] = transaction_train["UID"].map(uid_tag_dict)
    return operation_train, transaction_train

# Drop Duplicates
def DropDuplicates(operation_train, transaction_train, operation_test, transaction_test):
    print("Before Drop Duplicates: ")
    print("operation_train.shape: ", operation_train.shape)
    print("transaction_train.shape: ", transaction_train.shape)
    print("operation_test.shape: ", operation_test.shape)
    print("transaction_test.shape: ", transaction_test.shape)

    operation_train = operation_train.drop_duplicates()
    transaction_train = transaction_train.drop_duplicates()
    operation_test = operation_test.drop_duplicates()
    transaction_test = transaction_test.drop_duplicates()

    print("#" * 50)
    print("After Drop Duplicates: ")
    print("operation_train.shape: ", operation_train.shape)
    print("transaction_train.shape: ", transaction_train.shape)
    print("operation_test.shape: ", operation_test.shape)
    print("transaction_test.shape: ", transaction_test.shape)
    
    return operation_train, transaction_train, operation_test, transaction_test

def DataPreprocessing(operation_train, transaction_train, tag, operation_test, transaction_test):
    tag_test = CreateTagTest(operation_test, transaction_test)
    operation_train, transaction_train = MergeData(operation_train, transaction_train, tag)
    operation_test, transaction_test = MergeData(operation_test, transaction_test, tag_test)
    operation_train, transaction_train, operation_test, transaction_test = DropDuplicates(operation_train, transaction_train, operation_test, transaction_test)
    return operation_train, transaction_train, operation_test, transaction_test, tag_test

In [5]:
"""
    Step 2 : Data Exploration
    
    Categorical Columns : countplot, barplot
    Numerical Columns : regplot
    
"""

def DataExploration(operation_train, transaction_train, train_mode = False):
    
    print("Operation Countplot")
    plt.figure(figsize = (15, 10))
    i = 1
    for column in operation_train.columns:
        if operation_train[column].unique().shape[0] < 50:
            print("Plot " + column + "...")
            plt.subplot(2, 3, i)
            sns.countplot(x = column, data = operation_train)
            i += 1
    plt.suptitle("Operation Countplot")
    plt.show()

    if train_mode:
        print("#" * 100)
        print("Operation Barplot")
        plt.figure(figsize = (15, 10))
        i = 1
        for column in operation_train.columns:
            if operation_train[column].unique().shape[0] < 50 and column != "tag":
                print("Plot " + column + "...")
                plt.subplot(2, 2, i)
                sns.barplot(x = column, y = "tag", data = operation_train.sample(100000))
                i += 1
        plt.suptitle("Operation Barplot")
        plt.show()
    
    print("#" * 100)
    print("Transaction Countplot")
    plt.figure(figsize = (15, 10))
    i = 1
    for column in transaction_train.columns:
        if transaction_train[column].unique().shape[0] < 50:
            print("Plot " + column + "...")
            plt.subplot(2, 4, i)
            sns.countplot(x = column, data = transaction_train)
            i += 1
    plt.suptitle("Transaction Countplot")
    plt.show()

    if train_mode:
        print("#" * 100)
        print("Transaction Barplot")
        plt.figure(figsize = (15, 10))
        i = 1
        for column in transaction_train.columns:
            if transaction_train[column].unique().shape[0] < 50 and column != "tag":
                print("Plot " + column + "...")
                plt.subplot(2, 3, i)
                sns.barplot(x = column, y = "tag", data = transaction_train.sample(100000))
                i += 1
        plt.suptitle("transaction Barplot")
        plt.show()

# operation
# 时间日期：day, time
# 操作：mode, success
# 操作系统：os
# 版本：version
# 设备：device1, device2, device_code1, device_code2, device_code3
# ip：ip1, ip2, ip1_sub, ip2_sub
# mac：mac1, mac2
# wifi：wifi
# 地理位置：geo_code

# transaction
# 平台：channel
# 日期时间：day, time
# 资金：trans_amt, bal, amt_src1, amt_src2
# 商户：merchant, code1, code2
# 交易类型：trans_type1, trans_typ2
# 账户：acc_id1, acc_id2
# 设备：device_code1, device_code2, device_code3, device1, device2
# ip：ip1, ip1_sub
# mac：mac1
# 地理位置：geocode
# 营销活动：market_code, market_type

# operation countplot结论
# 1 日期分布较为均匀，操作记录较多的日期有1，8，15，22，29
# 2 绝大多数操作记录都成功了
# 3 操作系统分布不均，最多的是102，最少的是101和107
# 4 版本中，操作记录最多的是7.0.9和7.0.5
# 5 黑白样本记录不平衡，黑记录 / 白记录 = 137371 / 1096976 = 0.125；黑白样本比例不平衡，黑样本 / 白样本 = 4285 / 26894 = 0.16

# operation barplot结论
# 1 日期分布较为均匀，27附近的黑样本比例较高
# 2 操作成功的黑样本比例较高
# 3 操作系统为107的样本均为白样本，105的样本均为黑样本
# 4 版本为6.1.0，4.1.7的样本均为黑样本，6.5.0，7.0.0，6.6.3的黑样本比例较高，许多版本的样本均为白样本

In [6]:
"""
    Step 3 : Feature Engineering
    
    Feature Creation
    Feature Selection
    
"""

def FeatureCreation(data_train, tag):
    # Create hour, minute, second
    data_train["hour"] = data_train["time"].apply(lambda x : int(x[:2]))
    data_train["minute"] = data_train["time"].apply(lambda x : int(x[3:5]))
    data_train["second"] = data_train["time"].apply(lambda x : int(x[6:]))

    # Create data_count
    data_count = data_train.groupby("UID").count()["tag"].reset_index().rename(columns = {"tag" : "data_count"})
    data_features = pd.merge(tag, data_count, on = "UID", how = "left")

    # Create feature_count
    for column in data_train.columns:
        if column != "UID" and column != "tag":
            print("Create " + column + "_count...")
            feature_count = data_train.groupby("UID").nunique()[column].reset_index().rename(columns = {column : column + "_count"})
            data_features = pd.merge(data_features, feature_count, on = "UID", how = "left")

    # Create feature_value_count
    for column in data_train.columns:
        feature_values = data_train[column].unique()
        if column != "UID" and column != "tag" and feature_values.shape[0] < 50:
            for value in feature_values:
                if str(value) != "nan":
                    print("Create " + column + "_" + str(value) + "_count...")
                    feature_value = data_train[data_train[column] == value]
                    feature_value_count = feature_value.groupby("UID").count()[column].reset_index().rename(columns = {column : column + "_" + str(value) + "_count"})
                    data_features = pd.merge(data_features, feature_value_count, on = "UID", how = "left")

    return data_features

def FeatureMerge(operation_features, transaction_features):
    data_merge = pd.merge(operation_features, transaction_features, on = "UID", how = "outer")
    x_train = data_merge.drop(["UID", "Tag_x", "Tag_y"], axis = 1)
    y_train = data_merge["Tag_x"]
    return x_train, y_train

def FillNa(x_train):
    x_train = x_train.fillna(-1)
    return x_train

def FeatureSelection(x_train, y_train, model):
    sfm_model = SelectFromModel(model)
    sfm_model.fit(x_train, y_train)
    support = sfm_model.get_support()
    columns = list(range(len(support)))
    selected_columns = [col for col in range(len(columns)) if support[col]]
    selected_features = x_train.columns.values[selected_columns]
    return selected_features

def FeatureEngineering(operation_train, transaction_train, tag, selected_features = None):
    operation_features = FeatureCreation(operation_train, tag)
    transaction_features = FeatureCreation(transaction_train, tag)
    x_train, y_train = FeatureMerge(operation_features, transaction_features)
    x_train = FillNa(x_train)
    if selected_features is None:
        selected_features = FeatureSelection(x_train, y_train, GradientBoostingClassifier())
        x_train = x_train.loc[:, selected_features]
        return x_train, y_train, selected_features
    else:
#         x_train = x_train.loc[:, selected_features]
#         x_temp = pd.DataFrame()
#         for feature in selected_features:
#             if feature in x_train.columns:
#                 x_temp[feature] = x_train[feature]
#         return x_temp
        x_temp = pd.DataFrame(columns = selected_features)
        for feature in selected_features:
            if feature in x_train.columns:
                x_temp[feature] = x_train[feature]
        return x_temp

In [7]:
"""
    Step 4 : Model Optimization
    
    Models : lr, gbdt, xgb, lgbm
    
"""

def ModelOptimization(model, x_train, y_train):
    
    pass

In [8]:
"""
    Step 5 : Model Evaluation
    

"""

# Evaluation Function
def tpr_weight_function(y_true, y_predict):
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3

def ModelEvaluation(model, x_train, y_train):
    # roc_auc
    roc_auc = np.mean(cross_val_score(estimator = model, 
                                         X = x_train, 
                                         y = y_train, 
                                         scoring = "roc_auc", 
                                         cv = 3))
    print("roc_auc: ", roc_auc)

    # tpr_weight
    kf = KFold(n_splits = 3)
    model_scores = []
    for train_index, test_index in kf.split(x_train):
        x_tr, x_te = x_train[train_index], x_train[test_index]
        y_tr, y_te = y_train[train_index], y_train[test_index]
        model.fit(x_tr, y_tr)
        y_pred = model.predict(x_te)
        score = tpr_weight_function(y_te, y_pred)
        model_scores.append(score)
    print("tpr_weight: ", np.mean(model_scores))
    
    return roc_auc, tpr_weight

In [9]:
"""
    Step 6 : Model Ensembling

"""

def ModelEnsembling(models, x_train, y_train):
    pass

In [45]:
t_get_data_start = time.time()
operation_train, transaction_train, tag, operation_test, transaction_test = GetData(DATA_PATH)
operation_train, transaction_train, tag, operation_test, transaction_test = operation_train[:10000], transaction_train[:10000], tag, operation_test[:10000], transaction_test[:10000]
t_get_data_end = time.time()

t_data_preprocessing_start = time.time()
operation_train, transaction_train, operation_test, transaction_test, tag_test = DataPreprocessing(operation_train, transaction_train, tag, operation_test, transaction_test)
t_data_preprocessing_end = time.time()

t_data_exploration_start = time.time()
# DataExploration(operation_train, transaction_train, True)
t_data_exploration_end = time.time()

t_feature_engineering_start = time.time()
x_train, y_train, selected_features = FeatureEngineering(operation_train, transaction_train, tag)
x_test = FeatureEngineering(operation_test, transaction_test, tag_test, selected_features)
t_feature_engineering_end = time.time()

print("t_get_data: ", t_get_data_end - t_get_data_start)
print("t_data_preprocessing: ", t_data_preprocessing_end - t_data_preprocessing_start)
print("t_data_exploration: ", t_data_exploration_end - t_data_exploration_start)
print("t_feature_engineering: ", t_feature_engineering_end - t_feature_engineering_start)

g:\python\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


Before Drop Duplicates: 
operation_train.shape:  (10000, 21)
transaction_train.shape:  (10000, 28)
operation_test.shape:  (10000, 21)
transaction_test.shape:  (10000, 28)
##################################################
After Drop Duplicates: 
operation_train.shape:  (9934, 21)
transaction_train.shape:  (10000, 28)
operation_test.shape:  (9970, 21)
transaction_test.shape:  (10000, 28)
Create day_count...
Create mode_count...
Create success_count...
Create time_count...
Create os_count...
Create version_count...
Create device1_count...
Create device2_count...
Create device_code1_count...
Create device_code2_count...
Create device_code3_count...
Create mac1_count...
Create mac2_count...
Create ip1_count...
Create ip2_count...
Create wifi_count...
Create geo_code_count...
Create ip1_sub_count...
Create ip2_sub_count...
Create hour_count...
Create minute_count...
Create second_count...
Create day_30_count...
Create day_16_count...
Create day_8_count...
Create day_23_count...
Create day_2

Create version_7.0.7_count...
Create version_7.1.2_count...
Create version_7.0.2_count...
Create version_6.6.2_count...
Create version_7.0.0_count...
Create version_4.1.7_count...
Create version_6.6.3_count...
Create version_5.8.21_count...
Create version_6.0.4_count...
Create version_1.3.0_count...
Create version_6.0.5_count...
Create version_7.0.1_count...
Create version_1.0.0_count...
Create version_1.2.0_count...
Create version_1.1.0_count...
Create version_6.6.0_count...
Create version_6.5.0_count...
Create hour_15_count...
Create hour_16_count...
Create hour_21_count...
Create hour_6_count...
Create hour_0_count...
Create hour_13_count...
Create hour_8_count...
Create hour_10_count...
Create hour_9_count...
Create hour_20_count...
Create hour_19_count...
Create hour_12_count...
Create hour_14_count...
Create hour_3_count...
Create hour_18_count...
Create hour_11_count...
Create hour_17_count...
Create hour_22_count...
Create hour_7_count...
Create hour_23_count...
Create hour_1_c

In [46]:
x_train.head()

,device2_count_x,device_code1_count_x,device_code2_count_x,mac2_count,wifi_count,day_26_count_x,day_22_count_x,day_28_count_x,success_1.0_count,os_102_count,...,code1_count,trans_type1_count,device_code1_count_y,device_code2_count_y,device_code3_count_y,device2_count_y,geo_code_count_y,amt_src1_155c9e1c32bd0fa2_count,amt_src1_8c9987909b3e95a4_count,trans_type1_6d55c54c8b1056fb_count
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,2.0,2.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,1.0,1.0,2.0,1.0,1.0,-1.0,-1.0,-1.0,2.0,2.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [47]:
x_test.head()

,device2_count_x,device_code1_count_x,device_code2_count_x,mac2_count,wifi_count,day_26_count_x,day_22_count_x,day_28_count_x,success_1.0_count,os_102_count,...,code1_count,trans_type1_count,device_code1_count_y,device_code2_count_y,device_code3_count_y,device2_count_y,geo_code_count_y,amt_src1_155c9e1c32bd0fa2_count,amt_src1_8c9987909b3e95a4_count,trans_type1_6d55c54c8b1056fb_count
0,1.0,1.0,1.0,1.0,0.0,-1.0,-1.0,-1.0,1.0,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0
3,1.0,1.0,1.0,1.0,0.0,-1.0,1.0,-1.0,-1.0,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,5.0,2.0,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,-1.0,-1.0,-1.0
